![image](https://raw.githubusercontent.com/IBM/watson-machine-learning-samples/master/cloud/notebooks/headers/watsonx-Prompt_Lab-Notebook.png)

# Use Watsonx and `LangChain` to make a series of calls to a language model

#### Disclaimers

- Use only Projects and Spaces that are available in watsonx context.


## Notebook content

This notebook contains the steps and code to demonstrate Simple Sequential Chain using langchain integration with Watsonx models.

Some familiarity with Python is helpful. This notebook uses Python 3.10.


## Learning goal

The goal of this notebook is to demonstrate how to chain models available on watsonx.ai to generate a sequence of creating a random question on a given topic and an answer to that question and also to make the user friends with LangChain framework, using simple runnable sequences the WatsonxLLM class.


## Contents

This notebook contains the following parts:

- [Setup](#setup)
- [Foundation Models on Watsonx](#models)
- [WatsonxLLM interface](#watsonxllm)
- [Simple Sequential Chain experiment](#experiment)
- [Summary](#summary)

<a id="setup"></a>
## Set up the environment

Before you use the sample code in this notebook, you must perform the following setup tasks:

-  Create a <a href="https://console.ng.bluemix.net/catalog/services/ibm-watson-machine-learning/" target="_blank" rel="noopener no referrer">Watson Machine Learning (WML) Service</a> instance (a free plan is offered and information about how to create the instance can be found <a href="https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/ml-service-instance.html?context=analytics" target="_blank" rel="noopener no referrer">here</a>).

### watsonx.ai notebook environment

If you are running this from a Jupyter Notebook in watsonx.ai, then run the following code cell. 

**This may take a few seconds to complete.**

In [ ]:
# Install dependencies
import sys

!{sys.executable} -m pip install -U langchain-ibm | tail -n 1

### Non-watsonx.ai environment

If you are running this from a **outside** watsonx.ai, then run the following code cell. 

In [ ]:
import sys

!{sys.executable} -m pip install ibm-watsonx-ai | tail -n 1
!{sys.executable} -m pip install pydantic | tail -n 1
!{sys.executable} -m pip install langchain | tail -n 1
!{sys.executable} -m pip install -U langchain-ibm | tail -n 1

### Defining the WML credentials
This cell defines the WML credentials required to work with watsonx Foundation Model inferencing.

**Action:** Provide the IBM Cloud user API key. For details, see
[documentation](https://cloud.ibm.com/docs/account?topic=account-userapikey&interface=ui).

In [9]:
import getpass
from os import environ

try:
    REGION = environ["RUNTIME_ENV_REGION"]
except KeyError:
    # Set your region here if you are not running this notebook in the watsonx.ai Jupyter environment
    # us-south, eu-de, etc.
    REGION = "us-south"

credentials = {
    "url": "https://" + REGION + ".ml.cloud.ibm.com",
    "apikey": getpass.getpass("Please enter your WML api key (hit enter): "),
}

### Defining the project id
The Foundation Model requires project id that provides the context for the call. We will obtain the id from the project in which this notebook runs. Otherwise, please provide the project id.

In [10]:
from os import getenv

try:
    project_id = environ["PROJECT_ID"]
except KeyError:
    # Enter project ID here if not running this notebook in the watsonx.ai Jupyter environment
    project_id = "MY_PROJECT_ID"

<a id="models"></a>
## Foundation Models on `watsonx.ai`

#### List available models

All avaliable models are presented under `ModelTypes` class.

In [ ]:
from ibm_watsonx_ai.foundation_models.utils.enums import ModelTypes

print("Supported Models:")
for model in ModelTypes:
    print("-", model.name)

### Defining the model parameters

You might need to adjust model `parameters` for different models or tasks, to do so please refer to documentation under `GenTextParamsMetaNames` class.

**Action:** If any complications please refer to the [documentation](https://ibm.github.io/watson-machine-learning-sdk/).

In [23]:
from ibm_watsonx_ai.metanames import GenTextParamsMetaNames as GenParams
from ibm_watsonx_ai.foundation_models.utils.enums import DecodingMethods

parameters = {
    GenParams.DECODING_METHOD: DecodingMethods.SAMPLE,
    GenParams.MAX_NEW_TOKENS: 100,
    GenParams.MIN_NEW_TOKENS: 1,
    GenParams.TEMPERATURE: 0.5,
    GenParams.TOP_K: 50,
    GenParams.TOP_P: 1,
}

### Initialize the model
Initialize the `Model` class with previous set params.

In [24]:
from ibm_watsonx_ai.foundation_models import Model

flan_ul2_model = Model(
    model_id=ModelTypes.FLAN_UL2,
    params=parameters,
    credentials=credentials,
    project_id=project_id,
)

granite_20b_model = Model(
    model_id=ModelTypes.GRANITE_20B_MULTILINGUAL,
    credentials=credentials,
    project_id=project_id,
)

<a id="watsonxllm"></a>
## WatsonxLLM interface

`WatsonxLLM` is a wrapper around watsonx.ai models that provide chain integration around the models.

**Action:** For more details about `WatsonXLLM` check the [langchain-ibm package documentation](https://python.langchain.com/docs/integrations/llms/ibm_watsonx/).

### Initialize the `WatsonxLLM` class

In [25]:
from langchain_ibm import WatsonxLLM

# From WatsonX SDK Model object
flan_ul2_llm = flan_ul2_model.to_langchain()

# Brand-new WatsonxLLM object
granite_20b_llm = WatsonxLLM(
    model_id=ModelTypes.GRANITE_20B_MULTILINGUAL,
    apikey=credentials.get("apikey"),
    url=credentials.get("url", "https://us-south.ml.cloud.ibm.com"),
    project_id=project_id,
    params=parameters,
)

You can print all set data about the WatsonxLLM object using the `dict()` method.

In [ ]:
flan_ul2_llm.dict()

In [ ]:
granite_20b_llm.dict()

<a id="experiment"></a>
## Runnable Sequence experiment

To simplify creating custom workflows, LangChain has implemented a ["Runnable" protocol](https://python.langchain.com/v0.1/docs/expression_language/interface/). 

Many LangChain components implement the Runnable protocol, including chat models, LLMs, output parsers, retrievers, prompt templates, and more. 


allowing you to chain steps using the pipe operator (`|`), like so:

```
simple_chain = prompt | llm
```

The output of one step serves as the input for the following step.

The experiment will consist in generating a random question about any topic and answer the following question.

An object called `PromptTemplate` assists in generating prompts using a combination of user input, additional non-static data, and a fixed template string.

In our case we would like to create two `PromptTemplate` objects which will be responsible for creating a random question and answering it.

In [44]:
from langchain import PromptTemplate

gen_question = PromptTemplate(
    input_variables=["topic"],
    template="Generate a random question about {topic}: Question: ",
)
answer_question = PromptTemplate(
    input_variables=["question"],
    template="Answer the following question: {question}",
)

We would like to add functionality around language models using a simple runnable sequence:

```
chain = prompt | llm
```

`chain_q` chain formats the prompt template whose task is to generate random question, passes the formatted string to LLM and returns the LLM output.

In [ ]:
chain_q = gen_question | flan_ul2_llm

chain_q_res = chain_q.invoke(input="dogs")

print("LLM response:", chain_q_res)

`chain_answer_q` chain formats the prompt template whose task is to answer the question we got from `chain_q` chain, passes the formatted string to LLM and returns the LLM output.

In [ ]:
chain_answer_q = answer_question | granite_20b_llm

chain_answer_q_res = chain_answer_q.invoke(input=chain_q_res)

print("LLM response:", chain_answer_q_res)

Now, let's run the full sequence:

```
chain_q -> chain_answer_q
```

Which, if we break it down is something like:

```
gen_question | flan_ul2_llm -> answer_question | granite_20b_llm
```

In [ ]:
full_chain = chain_q | chain_answer_q

Generate random question and answer to topic.

In [ ]:
full_chain.invoke("Life")

<a id="summary"></a>
## Summary and next steps

 You successfully completed this notebook!.
 
 You learned how to use chain using custom llm WastonxLLM.
 
Check out our _[Online Documentation](https://ibm.github.io/watson-machine-learning-sdk/samples.html)_ for more samples, tutorials, documentation, how-tos, and blog posts. 

### Authors: 
 - **Mateusz Szewczyk**, Software Engineer at Watson Machine Learning.
 - **Josefina Casanova**, Engagement Lead, Build Lab Americas. Edited for L4 watsonx course. 2024

Copyright © 2024 IBM. This notebook and its source code are released under the terms of the MIT License.